# Joint Modeling Simulation - Parallel processing

# Imports

In [1]:
suppressPackageStartupMessages({
    library(JM)
    library(dplyr)
    library(nlme)
    library(survival)
    library(Matrix)
    library(MASS)
    library(future.apply)
})

# Simulation Settings

In [2]:
niters = 1:500
N = 250 # number of simulated individuals 
K = 10 # number of planned repeated measurements per subject, per outcome
cens_horiz = 10 # maximum follow-up time

# True Parameter Values

In [3]:
betas = c(-0.2, 0.3) # betas for longitudinal model
sigma.y = 0.5 # measurement error standard deviation
A = matrix(c(0.5, 0,
              0, 0.3),
            ncol=2) # covariance matrix for random effects
alpha = 0.6 # association parameter
h0 = 0.1 # constant baseline hazard

# Simulate Sequentially using for loops

In [ ]:
StartSim = Sys.time()
set.seed(555)
for (i in seq_along(niters)){
  # Longitudinal submodel ---------------------------------------------------
  # 1. Generate the id variable (each individual has K measurements)
  id = rep(1:N, each=K)
  
  # 2. Generate the times at which individuals have their longitudinal measurements
  msmt.times = c(replicate(N, c(0, cumsum(rexp(K-1)))))
  
  # 4. Create design matrices X and Z 
  # X: design matrix for fixed effects (should have as many columns as elements in "betas")
  # Z: design  matrix for random effects (should have as many columns as elements in "A")
  X = cbind(1, msmt.times)
  Ztemp = list()
  for (i in 1:(length(msmt.times)/K)){
    Ztemp[[i]] = cbind(1, msmt.times[(i+((K-1)*(i-1))):(K*i)])
  }
  
  Z = Reduce(bdiag, Ztemp)
  
  # 5. Simulate random effects
  # b: simulated random effects b using covariance matrix A 
  
  bMatrix = mvrnorm(N, mu=c(0,0), Sigma = A)
  bVector = split(t(bMatrix), ncol(bMatrix))[[1]]
  
  # 6. Simulate longitudinal responses
  eta.y = (X%*%betas + Z%*%bVector) #Xi'(t)*beta+zi'(t)*bi
  y = as.matrix(eta.y) + rnorm(K*N, 0, sigma.y) #~Normal(mean=eta.y, sd=sigma.y) 
  
  # 7. Add longitudinal responses to data set 
  dat = cbind(id, msmt.times, y)
  
  # Survival submodel ---------------------------------------------------------------------------
  # Integrate this function w.r.t "s" over the interval (0,t), where you want to solve for "t" 
  invS = function (t, u, i, betas, bMatrix) {
    #inverse function used to obtain survival time 
    #t: time 
    #u: draw from a uniform[0,1] distribution
    #i: patient id indicator 
    h = function (s) {
      #hazard function (based on survival and longitudinal submodels)
      m = (betas[1]+bMatrix[i,1]) + betas[2]*s + bMatrix[i,2]*s
      haz = h0*exp(alpha*m)
      return(haz)
    }
    integrate(h, lower = 0, upper = t)$value + log(u[i]) 
  }
  # 2. Solve for survival times 
  u = runif(N) #generate random U[0,1] random variable 
  trueTimes = numeric(N) #initiate vector of true survival times
  for (i in 1:N) {
    # Solve for true survival time (Hint: use "uniroot" function)
    # Wrap the uniroot function in "try" since for some individuals you will not return a valid 
    # survival time (we will consider those patients "cured" and assign them survival time of
    # infinity)
    Root = try({uniroot(invS, u=u, i=i, betas=betas, bMatrix=bMatrix, interval = c(0.01,10))}, silent = T) #Example: try(uniroot(, TRUE))
    # "Cure" patients have event time set to Infinity 
    trueTimes[i] = ifelse(inherits(Root, "try-error"), Inf,Root)
  }
  trueTimes = Reduce(rbind, trueTimes)
  rownames(trueTimes)=NULL
  # 3. Simulate censoring times 
  Ctimes = runif(N, 0.4, cens_horiz)
  
  # 4. Compute observed survival time and event indicator
  Time =  pmin(trueTimes, Ctimes)
  event =  ifelse(trueTimes <= Ctimes, 1, 0)
  
  
  # 5. Add in survival time and indicator to data set 
  dat = cbind(dat, rep(Time, each = 10), rep(event, each = 10))
  colnames(dat) = c(colnames(dat)[1:2],"y", "time", "event")
  # 6. Drop measurement times that occur after follow-up time 
  dat = dat[which(dat[,2] <= dat[,4]),]
  
  
  # 7. Create a data set with one row for each individual (for survival submodel)
  
  surv_dat = as.data.frame(dat) %>% group_by(id) %>% slice_head(n=1)
  surv_dat = surv_dat[,-2]
    
  # Create stop-start in surv data
  df_surv_interval = tmerge(surv_dat, surv_dat, id = id, death = event(time, event))
  
  # Add time varying covariates from the longi data
  df_interval = tmerge(df_surv_interval, as.data.frame(dat), id=id, Y = tdc(msmt.times, y))
  
  # Fit models --------------------------------------------------------------
  # 1. Fit joint model
  # use method="weibull-PH-aGH" to decrease computation time 
  startl = Sys.time()
  long_model = lme(y ~ msmt.times,
                   random = ~ msmt.times|id,
                   data = as.data.frame(dat))
  
  endl = Sys.time() - startl
  
  
  starts = Sys.time()
  surv_model = coxph(Surv(time,event) ~ 1,
                     x = TRUE,
                     data = surv_dat)
  
  ends = Sys.time() - starts
  
  
  startj = Sys.time()
  joint_model = jointModel(long_model, surv_model, timeVar = "msmt.times", method = "weibull-PH-aGH")
  
  endj = Sys.time() - startj
  
  # 2. Fit two-stage model
  startt = Sys.time()
  lme_pred = predict(long_model)
  surv_model_stage2 = coxph(Surv(tstart, tstop, death) ~ lme_pred,
                            data = df_interval)
  endt = Sys.time() - startt
  
  # Save appropriate coefficient estimates
  
  ## Joint model
  sigma = summary(joint_model)$sigma
  vcovA1 = summary(joint_model)$D[1]
  vcovA2 = summary(joint_model)$D[4]
  betas = joint_model$coefficients$betas
  alpha = joint_model$coefficients$alpha
  stdErrs = c(summary(joint_model)[1][[1]][,2], summary(joint_model)[2][[1]][,2][1:2])
  confints = cbind(confint(joint_model)[,1], confint(joint_model)[,3])
  
  ## Two stage model 
  twoStageAlpha = coef(surv_model_stage2)
  twoStageconfs = confint(surv_model_stage2)
  twoStageStdErrs = summary(surv_model_stage2)[7][[1]][3]
  
  outputs = list(LongSubModel = list(sigma = sigma,
                                     A1 = vcovA1,
                                     A2 = vcovA2),
                 JointModel = list(betas = betas,
                                   alpha = alpha,
                                   stdErrs = stdErrs,
                                   confints = confints),
                 TwoStageModel = list(Alpha = twoStageAlpha,
                                      confints = twoStageconfs,
                                      stdErrs = twoStageStdErrs),
                 Times = list(longitudinal = endl,
                              survival = ends,
                              joint = endj,
                              twostage = endt))
}
Sys.time() - StartSim

# Simulate Sequentially using LAPPLY

In [ ]:
StartSim = Sys.time()
set.seed(555)
sim = lapply(seq_along(niters), function(x){
  # Longitudinal submodel ---------------------------------------------------
  # 1. Generate the id variable (each individual has K measurements)
  id = rep(1:N, each=K)
  
  # 2. Generate the times at which individuals have their longitudinal measurements
  msmt.times = c(replicate(N, c(0, cumsum(rexp(K-1)))))
  
  # 4. Create design matrices X and Z 
  # X: design matrix for fixed effects (should have as many columns as elements in "betas")
  # Z: design  matrix for random effects (should have as many columns as elements in "A")
  X = cbind(1, msmt.times)
  Ztemp = list()
  for (i in 1:(length(msmt.times)/K)){
    Ztemp[[i]] = cbind(1, msmt.times[(i+((K-1)*(i-1))):(K*i)])
  }
  
  Z = Reduce(bdiag, Ztemp)
  
  # 5. Simulate random effects
  # b: simulated random effects b using covariance matrix A 
  
  bMatrix = mvrnorm(N, mu=c(0,0), Sigma = A)
  bVector = split(t(bMatrix), ncol(bMatrix))[[1]]
  
  # 6. Simulate longitudinal responses
  eta.y = (X%*%betas + Z%*%bVector) #Xi'(t)*beta+zi'(t)*bi
  y = as.matrix(eta.y) + rnorm(K*N, 0, sigma.y) #~Normal(mean=eta.y, sd=sigma.y) 
  
  # 7. Add longitudinal responses to data set 
  dat = cbind(id, msmt.times, y)
  
  # Survival submodel ---------------------------------------------------------------------------
  # Integrate this function w.r.t "s" over the interval (0,t), where you want to solve for "t" 
  invS = function (t, u, i, betas, bMatrix) {
    #inverse function used to obtain survival time 
    #t: time 
    #u: draw from a uniform[0,1] distribution
    #i: patient id indicator 
    h = function (s) {
      #hazard function (based on survival and longitudinal submodels)
      m = (betas[1]+bMatrix[i,1]) + betas[2]*s + bMatrix[i,2]*s
      haz = h0*exp(alpha*m)
      return(haz)
    }
    integrate(h, lower = 0, upper = t)$value + log(u[i]) 
  }
  # 2. Solve for survival times 
  u = runif(N) #generate random U[0,1] random variable 
  trueTimes = numeric(N) #initiate vector of true survival times
  for (i in 1:N) {
    # Solve for true survival time (Hint: use "uniroot" function)
    # Wrap the uniroot function in "try" since for some individuals you will not return a valid 
    # survival time (we will consider those patients "cured" and assign them survival time of
    # infinity)
    Root = try({uniroot(invS, u=u, i=i, betas=betas, bMatrix=bMatrix, interval = c(0,10))}, silent = T) #Example: try(uniroot(, TRUE))
    # "Cure" patients have event time set to Infinity 
    trueTimes[i] = ifelse(inherits(Root, "try-error"), Inf,Root)
  }
  trueTimes = Reduce(rbind, trueTimes)
  rownames(trueTimes)=NULL
  # 3. Simulate censoring times 
  Ctimes = runif(N, 0, cens_horiz)
  
  # 4. Compute observed survival time and event indicator
  Time =  pmin(trueTimes, Ctimes)
  event =  ifelse(trueTimes <= Ctimes, 1, 0)
  
  
  # 5. Add in survival time and indicator to data set 
  dat = cbind(dat, rep(Time, each = 10), rep(event, each = 10))
  colnames(dat) = c(colnames(dat)[1:2],"y", "time", "event")
  # 6. Drop measurement times that occur after follow-up time 
  dat = dat[which(dat[,2] <= dat[,4]),]
  
  
  # 7. Create a data set with one row for each individual (for survival submodel)
  
  surv_dat = as.data.frame(dat) %>% group_by(id) %>% slice_head(n=1)
  surv_dat = surv_dat[,-2]
    
  # Create stop-start in surv data
  df_surv_interval = tmerge(surv_dat, surv_dat, id = id, death = event(time, event))
  
  # Add time varying covariates from the longi data
  df_interval = tmerge(df_surv_interval, as.data.frame(dat), id=id, Y = tdc(msmt.times, y))
  
  # Fit models --------------------------------------------------------------
  # 1. Fit joint model
  # use method="weibull-PH-aGH" to decrease computation time 
  startl = Sys.time()
  long_model = lme(y ~ msmt.times,
                   random = ~ msmt.times|id,
                   data = as.data.frame(dat))
  
  endl = Sys.time() - startl
  
  
  starts = Sys.time()
  surv_model = coxph(Surv(time,event) ~ 1,
                     x = TRUE,
                     data = surv_dat)
  
  ends = Sys.time() - starts
  
  
  startj = Sys.time()
  joint_model = jointModel(long_model, surv_model, timeVar = "msmt.times", method = "weibull-PH-aGH")
  
  endj = Sys.time() - startj
  
  # 2. Fit two-stage model
  startt = Sys.time()
  lme_pred = predict(long_model)
  surv_model_stage2 = coxph(Surv(tstart, tstop, death) ~ lme_pred,
                            data = df_interval)
  endt = Sys.time() - startt
  
  # Save appropriate coefficient estimates
  
  ## Joint model
  sigma = summary(joint_model)$sigma
  vcovA1 = summary(joint_model)$D[1]
  vcovA2 = summary(joint_model)$D[4]
  betas = joint_model$coefficients$betas
  alpha = joint_model$coefficients$alpha
  stdErrs = c(summary(joint_model)[1][[1]][,2], summary(joint_model)[2][[1]][,2][1:2])
  confints = cbind(confint(joint_model)[,1], confint(joint_model)[,3])
  
  ## Two stage model 
  twoStageAlpha = coef(surv_model_stage2)
  twoStageconfs = confint(surv_model_stage2)
  twoStageStdErrs = summary(surv_model_stage2)[7][[1]][3]
  
  outputs = list(LongSubModel = list(sigma = sigma,
                                     A1 = vcovA1,
                                     A2 = vcovA2),
                 JointModel = list(betas = betas,
                                   alpha = alpha,
                                   stdErrs = stdErrs,
                                   confints = confints),
                 TwoStageModel = list(Alpha = twoStageAlpha,
                                      confints = twoStageconfs,
                                      stdErrs = twoStageStdErrs),
                 Times = list(longitudinal = endl,
                              survival = ends,
                              joint = endj,
                              twostage = endt))
})
Sys.time() - StartSim

ERROR: Error in tmerge(surv_dat, surv_dat, id = id, death = event(time, event)): found an ending time of 0, the default starting time of 0 is invalid


# Simulate in Parallel using Future LAPPLY

In [ ]:
StartSim = Sys.time()
plan(multisession)
sim = future_lapply(seq_along(niters), function(x){
  # Longitudinal submodel ---------------------------------------------------
  # 1. Generate the id variable (each individual has K measurements)
  id = rep(1:N, each=K)
  
  # 2. Generate the times at which individuals have their longitudinal measurements
  msmt.times = c(replicate(N, c(0, cumsum(rexp(K-1)))))
  
  # 4. Create design matrices X and Z 
  # X: design matrix for fixed effects (should have as many columns as elements in "betas")
  # Z: design  matrix for random effects (should have as many columns as elements in "A")
  X = cbind(1, msmt.times)
  Ztemp = list()
  for (i in 1:(length(msmt.times)/K)){
    Ztemp[[i]] = cbind(1, msmt.times[(i+((K-1)*(i-1))):(K*i)])
  }
  
  Z = Reduce(bdiag, Ztemp)
  
  # 5. Simulate random effects
  # b: simulated random effects b using covariance matrix A 
  
  bMatrix = mvrnorm(N, mu=c(0,0), Sigma = A)
  bVector = split(t(bMatrix), ncol(bMatrix))[[1]]
  
  # 6. Simulate longitudinal responses
  eta.y = (X%*%betas + Z%*%bVector) #Xi'(t)*beta+zi'(t)*bi
  y = as.matrix(eta.y) + rnorm(K*N, 0, sigma.y) #~Normal(mean=eta.y, sd=sigma.y) 
  
  # 7. Add longitudinal responses to data set 
  dat = cbind(id, msmt.times, y)
  
  # Survival submodel ---------------------------------------------------------------------------
  # Integrate this function w.r.t "s" over the interval (0,t), where you want to solve for "t" 
  invS = function (t, u, i, betas, bMatrix) {
    #inverse function used to obtain survival time 
    #t: time 
    #u: draw from a uniform[0,1] distribution
    #i: patient id indicator 
    h = function (s) {
      #hazard function (based on survival and longitudinal submodels)
      m = (betas[1]+bMatrix[i,1]) + betas[2]*s + bMatrix[i,2]*s
      haz = h0*exp(alpha*m)
      return(haz)
    }
    integrate(h, lower = 0, upper = t)$value + log(u[i]) 
  }
  # 2. Solve for survival times 
  u = runif(N) #generate random U[0,1] random variable 
  trueTimes = numeric(N) #initiate vector of true survival times
  for (i in 1:N) {
    # Solve for true survival time (Hint: use "uniroot" function)
    # Wrap the uniroot function in "try" since for some individuals you will not return a valid 
    # survival time (we will consider those patients "cured" and assign them survival time of
    # infinity)
    Root = try({uniroot(invS, u=u, i=i, betas=betas, bMatrix=bMatrix, interval = c(0.01,10))}, silent = T) #Example: try(uniroot(, TRUE))
    # "Cure" patients have event time set to Infinity 
    trueTimes[i] = ifelse(inherits(Root, "try-error"), Inf,Root)
  }
  trueTimes = Reduce(rbind, trueTimes)
  rownames(trueTimes)=NULL
  # 3. Simulate censoring times 
  Ctimes = runif(N, 0.4, cens_horiz)
  
  # 4. Compute observed survival time and event indicator
  Time =  pmin(trueTimes, Ctimes)
  event =  ifelse(trueTimes <= Ctimes, 1, 0)
  
  
  # 5. Add in survival time and indicator to data set 
  dat = cbind(dat, rep(Time, each = 10), rep(event, each = 10))
  colnames(dat) = c(colnames(dat)[1:2],"y", "time", "event")
  # 6. Drop measurement times that occur after follow-up time 
  dat = dat[which(dat[,2] <= dat[,4]),]
  
  
  # 7. Create a data set with one row for each individual (for survival submodel)
  
  surv_dat = as.data.frame(dat) %>% group_by(id) %>% slice_head(n=1)
  surv_dat = surv_dat[,-2]
    
  # Create stop-start in surv data
  df_surv_interval = tmerge(surv_dat, surv_dat, id = id, death = event(time, event))
  
  # Add time varying covariates from the longi data
  df_interval = tmerge(df_surv_interval, as.data.frame(dat), id=id, Y = tdc(msmt.times, y))
  
  # Fit models --------------------------------------------------------------
  # 1. Fit joint model
  # use method="weibull-PH-aGH" to decrease computation time 
  startl = Sys.time()
  long_model = lme(y ~ msmt.times,
                   random = ~ msmt.times|id,
                   data = as.data.frame(dat))
  
  endl = Sys.time() - startl
  
  
  starts = Sys.time()
  surv_model = coxph(Surv(time,event) ~ 1,
                     x = TRUE,
                     data = surv_dat)
  
  ends = Sys.time() - starts
  
  
  startj = Sys.time()
  joint_model = jointModel(long_model, surv_model, timeVar = "msmt.times", method = "weibull-PH-aGH")
  
  endj = Sys.time() - startj
  
  # 2. Fit two-stage model
  startt = Sys.time()
  lme_pred = predict(long_model)
  surv_model_stage2 = coxph(Surv(tstart, tstop, death) ~ lme_pred,
                            data = df_interval)
  endt = Sys.time() - startt
  
  # Save appropriate coefficient estimates
  
  ## Joint model
  sigma = summary(joint_model)$sigma
  vcovA1 = summary(joint_model)$D[1]
  vcovA2 = summary(joint_model)$D[4]
  betas = joint_model$coefficients$betas
  alpha = joint_model$coefficients$alpha
  stdErrs = c(summary(joint_model)[1][[1]][,2], summary(joint_model)[2][[1]][,2][1:2])
  confints = cbind(confint(joint_model)[,1], confint(joint_model)[,3])
  
  ## Two stage model 
  twoStageAlpha = coef(surv_model_stage2)
  twoStageconfs = confint(surv_model_stage2)
  twoStageStdErrs = summary(surv_model_stage2)[7][[1]][3]
  
  outputs = list(LongSubModel = list(sigma = sigma,
                                     A1 = vcovA1,
                                     A2 = vcovA2),
                 JointModel = list(betas = betas,
                                   alpha = alpha,
                                   stdErrs = stdErrs,
                                   confints = confints),
                 TwoStageModel = list(Alpha = twoStageAlpha,
                                      confints = twoStageconfs,
                                      stdErrs = twoStageStdErrs),
                 Times = list(longitudinal = endl,
                              survival = ends,
                              joint = endj,
                              twostage = endt))
}, future.seed = 555) # This is where we set the seed in future lapply
Sys.time() - StartSim

Time difference of 4.393695 mins

# Conclusion

The results for the conventional for loop took 29.5 minutes, we were able to speed things up a bit to 25.9 minutes using lapply, However, we are able to signifiicantly shave down the time to only 4.4 minutes to run the simulation when we run it in parallel. 